In [19]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import requests
import json
from config import gkey

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.colorado.com/colorado-breweries"


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [4]:
names_list = []
address_list = []
browser.visit(url)
time.sleep(1)
browser.find_by_tag("span[class*='icon icon--cross']").click()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
address_soup = soup.find_all("span", class_ = ("grid--result--address ng-binding"))
names_soup = soup.find_all("h3", class_ = ("grid--result--name ng-binding ng-scope"))
buttons = browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']")
buttons.click()

In [5]:
list_length = range(0, round(255/10))
for y in list_length:
    try: 
        if browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']") == True:
            buttons.click()
        else:
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            address_soup = soup.find_all("span", class_ = ("grid--result--address ng-binding"))
            names_soup = soup.find_all("h3", class_ = ("grid--result--name ng-binding ng-scope"))
            for x in names_soup:
                names_list.append(x.text)
            for x in address_soup:
                address_list.append(x.text)
            buttons = browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']")
            buttons.click()
    
    except:
        print("error")
display(address_list)
display(names_list)

error
error


['3201 Walnut Street, Boulder, CO 80301',
 '304 E Hopkins Ave, Aspen, CO 81611',
 '1338 South Valentia Street, Unit 100, Denver, CO 80247',
 '7803 Ralston Rd., Arvada, CO 80002',
 '182 Lusher Ct,, Frisco, CO 80443',
 '2516 49th Street, Boulder, CO 80301',
 '1700 Vine St, Denver, CO 80206',
 '111 N Market St, Cortez, CO 81321',
 '10595 Butte Drive, Longmont, CO 80504',
 '2670 E. Harmony Road, Fort Collins, CO 80525',
 '106 E. Jefferson Ave., Unit B, Hayden, CO 81639',
 '902 S Lipan St., Denver, CO 80223',
 '41290 B-2 & B-3 US HWY 6, Vail, CO 81657',
 '2070 County Road 12 South, Alamosa, CO 81101',
 '62  E. AGATE AVENUE, GRANBY, CO 80446',
 '501 Rose Street, Georgetown, CO 80444',
 '1560 E 2nd Ave, Durango, CO 81301',
 '225 S. Pine St., #G, Telluride, CO 81435',
 '2351 Busch Dr., Fort Collins, CO 80524',
 '6461 Highway 392, Windsor, CO 80550',
 '8855 W. 116th Circle Ste 4, Broomfield, CO 80021',
 '6560 Odell Pl. Unit D, Boulder, CO 80301',
 '10446 E Town Center Drive, Westminster, CO 800

['Twisted Pine Brewing Company',
 'Aspen Brewing Company',
 'Copper Kettle Brewing Co',
 'Yak & Yeti - Arvada',
 'Outer Range Brewing Company',
 'Stein Brewing',
 'Vine Street Pub & Brewery',
 'WildEdge Brewing Collective',
 'BrewHop Trolley',
 "BJ's Restaurant and Brewery",
 'Yampa Valley Brewing Co.',
 'Chain Reaction Brewing',
 'Vail Brewing Company',
 'The Colorado Farm Brewery',
 'Never Summer Brewing Co.',
 'Guanella Pass Brewing Company',
 'Animas Brewing Company',
 'Smuggler Union Restaurant & Brewery',
 'Budweiser Tour Center',
 'High Hops Brewery',
 '4 Noses Brewing Co.',
 'New Planet Brewery',
 "BJ's Restaurant and Brewhouse",
 'Call to Arms Brewing',
 '300 Suns Brewing',
 'Very Nice Brewing Company',
 'Mountain Sun Pub and Brewery',
 'Longs Peak Pub & Taphouse',
 'Mancos Brewing Company',
 'Alpine Dog Brewery',
 'Odell Brewing Company',
 'WeldWerks Brewing Company',
 'LowDown Brewery & Kitchen',
 'Brew Hop Trolley',
 'Pikes Peak Brewing',
 'Anheuser Busch Tour Center and Bi

In [6]:
len(address_list)

255

In [79]:
addresses = []
for x in address_list:
    addresses.append(x.replace(" ", "+"))

In [61]:
lats = []
lngs = []
g_url = "https://maps.googleapis.com/maps/api/geocode/json?address="
for x in addresses:
    try:
        response = requests.get(f'{g_url}{x}&key={gkey}')
        places_data = response.json()
        lats.append(places_data["results"][0]['geometry']['location']['lat'])
        lngs.append(places_data["results"][0]['geometry']['location']['lng'])
    except:
        lats.append("NaN")
        lngs.append("NaN")

In [78]:
breweries_df = pd.DataFrame(names_list, address_list)
breweries_df["Latitudes"] = lats
breweries_df["Longitudes"] = lngs
breweries_df = breweries_df.reset_index()
breweries_df = breweries_df.rename(columns = ({0 : "Brewery", "index": "Address"}))
breweries_df = breweries_df.set_index("Brewery")
breweries_df = breweries_df.dropna()
cleaned_breweries = breweries_df[breweries_df["Longitudes"] != "NaN"]
cleaned_breweries.to_csv("cleaned_breweries.csv")
cleaned_breweries

,Address,Latitudes,Longitudes
Brewery,,,
Twisted Pine Brewing Company,"3201 Walnut Street, Boulder, CO 80301",40.0208,-105.251
Aspen Brewing Company,"304 E Hopkins Ave, Aspen, CO 81611",39.1903,-106.82
Copper Kettle Brewing Co,"1338 South Valentia Street, Unit 100, Denver, ...",39.6924,-104.891
Yak & Yeti - Arvada,"7803 Ralston Rd., Arvada, CO 80002",39.8024,-105.084
Outer Range Brewing Company,"182 Lusher Ct,, Frisco, CO 80443",39.5893,-106.099
...,...,...,...
Avalanche Brewing Co.,"1067 Blair St., Silverton, CO 81433",37.8103,-107.665
Cb & Potts,"261 Kaycee Case Pl, Colorado Springs, CO 80921",39.0215,-104.826
De Steeg Brewing Co.,"4342 Tennyson St., Denver, CO 80212",39.7761,-105.044
